# 1. Install libraries a/n

In [96]:
# Install required libraries (uncomment and run if not already installed)
# !pip install pandas neo4j
# !pip install beautifulsoup4 lxml 
# !pip install requests tqdm

# Import necessary libraries
import pandas as pd
from neo4j import GraphDatabase


## 2. Create Table of all unique survey IDs
- The first step is to create our survey table
- This will be the 'Central Table' for organizing all of our information

### 2.1 Loading Census API Dataset info
- File was obtained from `https://api.census.gov/data.html`

In [122]:
# Import required libraries
from bs4 import BeautifulSoup
import pandas as pd

# Specify the path to your HTML file
html_file_path = './data/CensusDataAPI_data.html'  # Update this path as needed

# Define the columns that contain URLs
url_columns = [
    'Geography List',
    'Variable List',
    'Group List',
    'SortList',
    'Examples',
    'Developer Documentation',
    'API Base URL'
]

# Load and parse the HTML file
with open(html_file_path, 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'lxml')

# Find all tables in the HTML
tables = soup.find_all('table')
print(f"Number of tables found: {len(tables)}")

# Check if at least one table is found
if not tables:
    raise ValueError("No tables found in the HTML file.")

# Assuming your data is in the first table; adjust the index if necessary
table = tables[0]

# Extract table headers
headers = [th.get_text(strip=True) for th in table.find_all('th')]
print(f"Headers: {headers}")

# Initialize a list to store each row's data
rows = []

# Iterate over each row in the table (skip header row)
for tr in table.find_all('tr')[1:]:
    cells = tr.find_all(['td', 'th'])
    row_data = {}
    for idx, cell in enumerate(cells):
        # Get the header for the current cell
        header = headers[idx] if idx < len(headers) else f'Column_{idx+1}'

        if header in url_columns:
            # Extract all href attributes from <a> tags
            links = cell.find_all('a')
            urls = [link.get('href') for link in links if link.get('href')]

            # If no <a> tags, check if the cell contains a plain URL
            if not urls:
                cell_text = cell.get_text(strip=True)
                if cell_text.startswith('http://') or cell_text.startswith('https://'):
                    urls = [cell_text]

            # Join multiple URLs with '; ' or set as None if no URLs found
            row_data[header] = '; '.join(urls) if urls else None
        else:
            # For other columns, store the text
            row_data[header] = cell.get_text(strip=True)
    rows.append(row_data)

# Create a DataFrame from the extracted data
df_census = pd.DataFrame(rows)

# Display the first few rows of the DataFrame
df_census.head()


Number of tables found: 1
Headers: ['Title', 'Description', 'Vintage', 'Dataset Name', 'Dataset Type', 'Geography List', 'Variable List', 'Group List', 'SortList', 'Examples', 'Developer Documentation', 'API Base URL']


,Title,Description,Vintage,Dataset Name,Dataset Type,Geography List,Variable List,Group List,SortList,Examples,Developer Documentation,API Base URL
0,1648 datasets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986 County Business Patterns: Business Patterns,County Business Patterns (CBP) is an annual se...,1986,cbp,Aggregate,http://api.census.gov/data/1986/cbp/geography....,http://api.census.gov/data/1986/cbp/variables....,http://api.census.gov/data/1986/cbp/groups.html,http://api.census.gov/data/1986/cbp/sorts.html,http://api.census.gov/data/1986/cbp/examples.html,http://www.census.gov/developer/,http://api.census.gov/data/1986/cbp
2,1987 County Business Patterns: Business Patterns,County Business Patterns (CBP) is an annual se...,1987,cbp,Aggregate,http://api.census.gov/data/1987/cbp/geography....,http://api.census.gov/data/1987/cbp/variables....,http://api.census.gov/data/1987/cbp/groups.html,http://api.census.gov/data/1987/cbp/sorts.html,http://api.census.gov/data/1987/cbp/examples.html,http://www.census.gov/developer/,http://api.census.gov/data/1987/cbp
3,1988 County Business Patterns: Business Patterns,County Business Patterns (CBP) is an annual se...,1988,cbp,Aggregate,http://api.census.gov/data/1988/cbp/geography....,http://api.census.gov/data/1988/cbp/variables....,http://api.census.gov/data/1988/cbp/groups.html,http://api.census.gov/data/1988/cbp/sorts.html,http://api.census.gov/data/1988/cbp/examples.html,http://www.census.gov/developer/,http://api.census.gov/data/1988/cbp
4,1989 County Business Patterns: Business Patterns,County Business Patterns (CBP) is an annual se...,1989,cbp,Aggregate,http://api.census.gov/data/1989/cbp/geography....,http://api.census.gov/data/1989/cbp/variables....,http://api.census.gov/data/1989/cbp/groups.html,http://api.census.gov/data/1989/cbp/sorts.html,http://api.census.gov/data/1989/cbp/examples.html,http://www.census.gov/developer/,http://api.census.gov/data/1989/cbp


### 2.2 Parsing data set name column
- There is hierarchial information embedded in this column, potenially use for the graph database createion
- However, just in case, we're going to extract that, and grab the 'month' as time data we may want to use as a relationship.

In [123]:
# Define the delimiter used in the "Dataset Name" column
delimiter = '›'

# Split the "Dataset Name" into hierarchical levels, maximum of 3 splits (4 parts)
hierarchy_split = df_census['Dataset Name'].str.split(delimiter, n=3, expand=True)

# Rename the new columns based on hierarchy levels
hierarchy_split = hierarchy_split.rename(columns={
    0: 'Survey',
    1: 'Subtype1',
    2: 'Subtype2',
    3: 'Subtype3'
})

# Concatenate the new hierarchy columns with the original DataFrame
df_census = pd.concat([df_census, hierarchy_split], axis=1)

# Display the first few rows after parsing
df_census[['Survey', 'Subtype1', 'Subtype2', 'Subtype3', 'Vintage']].head(7)


,Survey,Subtype1,Subtype2,Subtype3,Vintage
0,NaN,NaN,NaN,NaN,NaN
1,cbp,None,None,None,1986
2,cbp,None,None,None,1987
3,cbp,None,None,None,1988
4,cbp,None,None,None,1989
5,cps,basic,apr,None,1989
6,cps,basic,aug,None,1989


### 2.3 Extract Month from subtype columns

In [124]:
# Define a list of month abbreviations for identification (lowercase for matching)
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
          'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Function to identify and extract month from subtype columns
def extract_month(row):
    for subtype_col in ['Subtype1', 'Subtype2', 'Subtype3']:
        if pd.notnull(row[subtype_col]):
            if row[subtype_col].strip().lower() in months:
                return row[subtype_col].strip().capitalize()
    return None

# Apply the function to create a new 'Month' column
df_census['Month'] = df_census.apply(extract_month, axis=1)

# Remove the month from the subtype columns to avoid duplication
for subtype_col in ['Subtype1', 'Subtype2', 'Subtype3']:
    df_census[subtype_col] = df_census[subtype_col].apply(
        lambda x: None if pd.notnull(x) and x.strip().lower() in months else x
    )

# Display the first few rows after extracting 'Month'
df_census[['Survey', 'Subtype1', 'Subtype2', 'Subtype3', 'Month', 'Vintage']].head()


,Survey,Subtype1,Subtype2,Subtype3,Month,Vintage
0,NaN,NaN,NaN,NaN,None,NaN
1,cbp,None,None,None,None,1986
2,cbp,None,None,None,None,1987
3,cbp,None,None,None,None,1988
4,cbp,None,None,None,None,1989


### 2.4 Get a unique ID for each row
1. Use the identifier from the API URL which is a JSON file that contains a unique identifier (aka KEY) for each dataset.
2. Why Use identifier as the Key:
- Uniqueness: The identifier provides a unique reference for each dataset, ensuring there are no duplicates.
- Consistency: Using a standardized key helps in linking data across different sources and maintaining data integrity within your knowledge graph.
- Efficiency: It simplifies data retrieval and relationships within the knowledge graph.

#### 2.4.1 First, drop the first row contains the number of records and a bunch of na values

In [133]:
# Removeing first row where all elements are NaN
# Before removing first row
print("Before dropping row:")
print(df_census.head(1))

# Removing first row
df_census = df_census.drop(0).reset_index(drop=True)

# After removing first row
print("\nAfter dropping row:")
print(df_census.head(1))

Before dropping row:
           Title Description Vintage Dataset Name Dataset Type Geography List  \
0  1648 datasets         NaN     NaN          NaN          NaN            NaN   

  Variable List Group List SortList Examples Developer Documentation  \
0           NaN        NaN      NaN      NaN                     NaN   

  API Base URL Survey Subtype1 Subtype2 Subtype3 Month identifier id_name  
0          NaN    NaN      NaN      NaN      NaN  None       None    None  

After dropping row:
                                              Title  \
0  1986 County Business Patterns: Business Patterns   

                                         Description Vintage Dataset Name  \
0  County Business Patterns (CBP) is an annual se...    1986          cbp   

  Dataset Type                                     Geography List  \
0    Aggregate  http://api.census.gov/data/1986/cbp/geography....   

                                       Variable List  \
0  http://api.census.gov/data/1986/cb

#### 2.4.2 Extracting the Identifier Field to create key column SurveyID
- Robust error handling was added do account for network issues

In [135]:
# Import required libraries
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import pandas as pd
from tqdm import tqdm  # For progress bar

# Remove rows where all elements are NaN
df_census.dropna(how='all', inplace=True)
df_census.reset_index(drop=True, inplace=True)

# Display current DataFrame columns to confirm
print("Current DataFrame Columns:")
print(df_census.columns.tolist())

# Step 1: Set up a session with retries (move this outside the function)
session = requests.Session()
retries = Retry(
    total=5,  # Total number of retries
    backoff_factor=1,  # Time to wait between retries (exponential backoff)
    status_forcelist=[500, 502, 503, 504],  # Retry on these HTTP status codes
    allowed_methods=['GET']  # Use 'allowed_methods' instead of 'method_whitelist'
)
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Step 2: Define a Function to Extract 'identifier' and 'SurveyID' from a Single URL with Retries

def extract_identifier(url, session):
    """
    Fetches JSON data from the given URL and extracts the 'identifier' and 'SurveyID'.
    
    Parameters:
        url (str): The API Base URL pointing to the JSON file.
        session (requests.Session): The session object with retry strategy.
    
    Returns:
        tuple: (identifier, SurveyID) where:
               - identifier (str or None): The full identifier URL.
               - SurveyID (str or None): The part of the identifier after '/id/'.
    """
    try:
        # Ensure the URL is a valid string
        if not isinstance(url, str) or pd.isna(url):
            print("Invalid URL encountered.")
            return None, None

        # Fetch the JSON data from the URL
        response = session.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        json_data = response.json()
        
        # Extract the 'identifier' field from the first dataset
        dataset_list = json_data.get('dataset', [])
        if isinstance(dataset_list, list) and len(dataset_list) > 0:
            identifier = dataset_list[0].get('identifier', None)
        else:
            identifier = None
        
        # Extract the 'SurveyID' by splitting the 'identifier' at '/id/'
        if identifier and '/id/' in identifier:
            SurveyID = identifier.split('/id/')[-1]
        else:
            SurveyID = None
        
        return identifier, SurveyID
    
    except requests.exceptions.Timeout:
        print(f"Timeout error occurred while fetching {url}")
        return None, None
    except requests.exceptions.ConnectionError:
        print(f"Connection error occurred while fetching {url}")
        return None, None
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred while fetching {url}: {e}")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching {url}: {e}")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred while processing {url}: {e}")
        return None, None

# Step 3: Apply the Extraction Function to All Rows

# Initialize lists to store the results
identifiers = []
survey_ids = []

# Iterate over each API Base URL and extract identifiers
for url in tqdm(df_census['API Base URL'], desc="Fetching identifiers"):
    identifier, SurveyID = extract_identifier(url, session)
    identifiers.append(identifier)
    survey_ids.append(SurveyID)

# Step 4: Add the Extracted Data to the DataFrame

df_census['identifier'] = identifiers
df_census['SurveyID'] = survey_ids

# Display the first few rows to verify the extraction
print("\nAfter extracting 'identifier' and 'SurveyID':")
print(df_census[['Survey', 'Subtype1', 'Subtype2', 'Subtype3', 'Vintage', 'identifier', 'SurveyID']].head())

# Step 5: Handle Missing Identifiers

# Check for missing identifiers
missing_identifiers = df_census['identifier'].isnull().sum()
print(f"\nNumber of missing identifiers: {missing_identifiers}")

# If there are missing identifiers, display them and save to a separate CSV for manual correction
if missing_identifiers > 0:
    print("\nRows with missing identifiers:")
    missing_identifiers_df = df_census[df_census['identifier'].isnull()]
    print(missing_identifiers_df[['Survey', 'API Base URL']].head())
    
    # Save these rows to a separate CSV file
    missing_identifiers_df.to_csv('./data/missing_identifiers.csv', index=False)
    print("\nSaved rows with missing identifiers to 'missing_identifiers.csv' for manual correction.")
else:
    print("All identifiers were successfully extracted.")

# Step 6: Save the Updated DataFrame for Backup

# Save the updated DataFrame to a CSV file
output_file = './data/df_census_with_identifiers.csv'
df_census.to_csv(output_file, index=False)
print(f"\nUpdated DataFrame with 'identifier' and 'SurveyID' successfully saved to {output_file}")


Current DataFrame Columns:
['Title', 'Description', 'Vintage', 'Dataset Name', 'Dataset Type', 'Geography List', 'Variable List', 'Group List', 'SortList', 'Examples', 'Developer Documentation', 'API Base URL', 'Survey', 'Subtype1', 'Subtype2', 'Subtype3', 'Month', 'identifier', 'id_name']


Fetching identifiers: 100%|█████████████████| 1648/1648 [02:27<00:00, 11.17it/s]



After extracting 'identifier' and 'SurveyID':
  Survey Subtype1 Subtype2 Subtype3 Vintage  \
0    cbp     None     None     None    1986   
1    cbp     None     None     None    1987   
2    cbp     None     None     None    1988   
3    cbp     None     None     None    1989   
4    cps    basic     None     None    1989   

                                      identifier        SurveyID  
0          http://api.census.gov/data/id/CBP1986         CBP1986  
1          http://api.census.gov/data/id/CBP1987         CBP1987  
2          http://api.census.gov/data/id/CBP1988         CBP1988  
3          http://api.census.gov/data/id/CBP1989         CBP1989  
4  https://api.census.gov/data/id/CPSBASIC198904  CPSBASIC198904  

Number of missing identifiers: 0
All identifiers were successfully extracted.

Updated DataFrame with 'identifier' and 'SurveyID' successfully saved to ./data/df_census_with_identifiers.csv


#### 2.4.3 Manually fixing errors (Only if needed)
- Recommended: Rerun! Maybe wait if there is a network issue.
- Manualy fixing is an acceptable strategy, but prone to human error.
- robust error handling was added to avoid errors, but stuff happens.


## 3. Create External Tables and Linkages for Survey Table
1. Several columns have URLs that link to other tables: 'Geography List', 'Variable List', 'Group List', 'SortList', 'Examples'
2. In order to avoid issues with data scraping, collect a copy of the data with linkages back to SurveyID
3. Variables and Group have more complex structures that will require additional processing to capture the information
4. 'SortList' is believed to be a parameter and probably doesn't have any info, but we'll look anyways

> We will programatically create the graph database from these tables. The CSV files are an intermediary step, but one done out of practical necessity to avoid retrieval issues and create a solid foundation to work from with all data local to the compute.

Testing identifier extraction with URL: http://api.census.gov/data/1986/cbp

Fetched JSON data:
{'@context': 'https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld',
 '@id': 'http://api.census.gov/data/1986/cbp.json',
 '@type': 'dcat:Catalog',
 'conformsTo': 'https://project-open-data.cio.gov/v1.1/schema',
 'dataset': [{'@type': 'dcat:Dataset',
              'accessLevel': 'public',
              'bureauCode': ['006:07'],
              'c_dataset': ['cbp'],
              'c_documentationLink': 'http://www.census.gov/developer/',
              'c_examplesLink': 'http://api.census.gov/data/1986/cbp/examples.json',
              'c_geographyLink': 'http://api.census.gov/data/1986/cbp/geography.json',
              'c_groupsLink': 'http://api.census.gov/data/1986/cbp/groups.json',
              'c_isAggregate': True,
              'c_isAvailable': True,
              'c_sorts_url': 'http://api.census.gov/data/1986/cbp/sorts.json',
              'c_tagsLink': 'http://api.census.gov/d

In [39]:
# File paths for variables metadata (by decade)
variables_file_paths = {
    '1980s': "./data/census_metadata_1980s.csv",
    '1990s': "./data/census_metadata_1990s.csv",
    '2000s': "./data/census_metadata_2000s.csv",
    '2010s': "./data/census_metadata_2010s.csv",
    '2020s': "./data/census_metadata_2020s.csv",
    'UNKN': "./data/census_metadata_Unknown.csv"
}

# Helper function for handling 'Unknown' years
def handle_unknown_years(df):
    """Replaces 'Unknown' in the 'year' column with 9999 for consistency."""
    if 'year' in df.columns:
        df['year'] = df['year'].astype(str)  # Ensure 'year' is treated as a string
        df['year'] = df['year'].replace('Unknown', '9999')  # Replace 'Unknown' with 9999
    return df

# Function to load datasets metadata
def load_datasets_metadata(file_path):
    """
    Load the combined datasets metadata file.
    Args:
        file_path (str): Path to the datasets metadata CSV file.
    Returns:
        pd.DataFrame: Loaded datasets metadata.
    """
    try:
        datasets_metadata = pd.read_csv(file_path)
        print("Datasets Metadata Loaded:")
        display(datasets_metadata.head(1))
        return datasets_metadata
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return pd.DataFrame()  # Return empty DataFrame if file not found

# Function to load variables metadata
def load_variables_metadata(file_paths):
    """
    Load and combine variables metadata files for all decades.
    Args:
        file_paths (dict): Dictionary of decade to file path mappings.
    Returns:
        pd.DataFrame: Combined variables metadata.
    """
    all_variables_metadata = []
    for decade, file_path in file_paths.items():
        try:
            # Load metadata for the current decade
            variables_metadata = pd.read_csv(file_path, low_memory=False)
            print(f"Variables Metadata for {decade} Loaded:")
            display(variables_metadata.head(1))

            # Handle 'Unknown' year values for the UNKN dataset
            if decade == 'UNKN':
                variables_metadata = handle_unknown_years(variables_metadata)

            all_variables_metadata.append(variables_metadata)
        except FileNotFoundError:
            print(f"Error: File not found for {decade}: {file_path}")
            continue

    # Combine all variables metadata into one dataframe
    combined_variables_metadata = pd.concat(all_variables_metadata, ignore_index=True)

    # Ensure the 'year' column is numeric and handle invalid values
    if 'year' in combined_variables_metadata.columns:
        combined_variables_metadata['year'] = pd.to_numeric(combined_variables_metadata['year'], errors='coerce')
        combined_variables_metadata['year'] = combined_variables_metadata['year'].fillna(9999)  # Replace NaNs with 9999
        combined_variables_metadata['year'] = combined_variables_metadata['year'].astype(int)  # Convert to integer type

    return combined_variables_metadata

# File path for the datasets metadata
datasets_file_path = "./data/census_datasets_metadata_cleaned.csv"

# Load both datasets and variables metadata
datasets_metadata = load_datasets_metadata(datasets_file_path)
combined_variables_metadata = load_variables_metadata(variables_file_paths)

# Display loaded datasets and variables metadata
print("Combined Datasets Metadata:")
display(datasets_metadata.head(1))

print("Combined Variables Metadata:")
display(combined_variables_metadata.head(1))

# Save the cleaned data to new CSV files (optional)
combined_variables_metadata.to_csv('./data/combined_variables_metadata.csv', index=False)
datasets_metadata.to_csv('./data/cleaned_datasets_metadata.csv', index=False)

print("Data ingest completed.")


Datasets Metadata Loaded:


,dataset_name,year,title,description,identifier,contact,access_level,modified,publisher,references,keywords
0,cps,1994,Jun 1994 Current Population Survey: Basic Monthly,"To provide estimates of employment, unemployme...",https://api.census.gov/data/id/CPSBASIC199406,CPS Staff,public,2019-10-09 15:05:36.0,U.S. Census Bureau,https://www.census.gov/developers/,census


Variables Metadata for 1980s Loaded:


,dataset_name,year,title,variable_name,label,concept,predicateType,group,limit,attributes,decade
0,cbp,1986,1986 County Business Patterns: Business Patterns,for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,NaN,0,NaN,1980s


Variables Metadata for 1990s Loaded:


...

Combined Datasets Metadata:


,dataset_name,year,title,description,identifier,contact,access_level,modified,publisher,references,keywords
0,cps,1994,Jun 1994 Current Population Survey: Basic Monthly,"To provide estimates of employment, unemployme...",https://api.census.gov/data/id/CPSBASIC199406,CPS Staff,public,2019-10-09 15:05:36.0,U.S. Census Bureau,https://www.census.gov/developers/,census


Combined Variables Metadata:


,dataset_name,year,title,variable_name,label,concept,predicateType,group,limit,attributes,decade
0,cbp,1986,1986 County Business Patterns: Business Patterns,for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,NaN,0,NaN,1980s


Data ingest completed.


# 3. Clean the Data

## **Data Structuring and Relationship Design**

### **Overview**
The goal of structuring the data is to create a hierarchical and navigable knowledge graph that balances usability and precision. This involves grouping repeated surveys into logical parent-child relationships while maintaining links to their associated variables and years.

### **Key Design Decisions**
1. **Parent-Child Relationships for Datasets:**
   - **Why:** Many surveys are conducted multiple times per year, with variations in their metadata. A parent-child hierarchy simplifies navigation for users querying high-level information while retaining granularity for specific queries.
   - **How:** We use the `parent_dataset` field to represent the high-level grouping (e.g., `cps` for the Current Population Survey) and `dataset_name` for specific instances (e.g., `cps/basic/jan` for the January survey).

2. **Linking Variables to Child Datasets:**
   - **Why:** Each dataset instance includes specific variables. Establishing this connection allows users to query datasets for their variables or find which datasets a variable belongs to.
   - **How:** We create `Variable` nodes linked to `ChildDataset` nodes via an `INCLUDES` relationship.

3. **Year-Based Relationships:**
   - **Why:** Many datasets are time-specific. Linking datasets to their respective years ensures queries can filter datasets by year and handle temporal questions like, "What data is available for 1986?"
   - **How:** We create `Year` nodes and connect them to `ChildDataset` nodes via a `BELONGS_TO_YEAR` relationship.

### **Graph Schema**
Here is the schema we use to represent the relationships:
- **ParentDataset**: Represents high-level groupings of surveys (e.g., `cps`, `cbp`).
  - **Relationships:**
    - `PARENT_OF` → `ChildDataset`
- **ChildDataset**: Represents individual survey instances (e.g., `cps/basic/jan`).
  - **Relationships:**
    - `INCLUDES` → `Variable`
    - `BELONGS_TO_YEAR` → `Year`
- **Variable**: Represents specific data variables (e.g., `employment_status`).
- **Year**: Represents the temporal context for datasets (e.g., `1986`).

### **Why This Structure?**
This design ensures:
- **Scalability**: Easily add new datasets, variables, and years.
- **Usability**: Queries can target high-level overviews or specific details.
- **Flexibility**: Supports both general and granular user queries.


In [78]:
import pandas as pd

def restructure_for_graph(datasets_metadata, variables_metadata):
    """
    Restructure datasets and variables metadata for hierarchical knowledge graph.
    Args:
        datasets_metadata (pd.DataFrame): Metadata for datasets.
        variables_metadata (pd.DataFrame): Metadata for variables.
    Returns:
        dict: Nodes and relationships for building the graph.
    """
    # Step 1: Extract unique surveys
    surveys = datasets_metadata['dataset_name'].str.split('/').str[0].unique()
    survey_nodes = pd.DataFrame({'survey': surveys})
    print(f"Survey Nodes: {survey_nodes.shape[0]}")

    # Step 2: Create dataset nodes
    datasets_metadata['parent_survey'] = datasets_metadata['dataset_name'].str.split('/').str[0]
    datasets_metadata['month'] = datasets_metadata['title'].str.extract(r'(\bJan|\bFeb|\bMar|\bApr|\bMay|\bJun|\bJul|\bAug|\bSep|\bOct|\bNov|\bDec)', expand=False)

    dataset_nodes = datasets_metadata[['parent_survey', 'year', 'month', 'title', 'description']].drop_duplicates()
    dataset_nodes['dataset_id'] = dataset_nodes.apply(lambda x: f"{x['parent_survey']}_{x['year']}_{x['month'] or 'Annual'}", axis=1)
    print(f"Dataset Nodes: {dataset_nodes.shape[0]}")

    # Step 3: Create variable nodes
    variables_metadata['parent_survey'] = variables_metadata['dataset_name'].str.split('/').str[0]
    variable_nodes = variables_metadata[['parent_survey', 'dataset_name', 'year', 'variable_name', 'label', 'concept']].drop_duplicates()
    print(f"Variable Nodes: {variable_nodes.shape[0]}")

    # Step 4: Create relationships
    survey_to_dataset = dataset_nodes[['parent_survey', 'dataset_id']]
    dataset_to_variable = variable_nodes[['dataset_name', 'variable_name']]
    print(f"Survey-to-Dataset Relationships: {survey_to_dataset.shape[0]}")
    print(f"Dataset-to-Variable Relationships: {dataset_to_variable.shape[0]}")

    return {
        'survey_nodes': survey_nodes,
        'dataset_nodes': dataset_nodes,
        'variable_nodes': variable_nodes,
        'relationships': {
            'survey_to_dataset': survey_to_dataset,
            'dataset_to_variable': dataset_to_variable
        }
    }

# Reload and clean metadata
print("Reloading datasets_metadata and variables_metadata...")
datasets_metadata = pd.read_csv('./data/census_datasets_metadata.csv')
variables_metadata = pd.read_csv('./data/combined_variables_metadata.csv')
print("Metadata loaded.")

# Restructure for graph
graph_data = restructure_for_graph(datasets_metadata, variables_metadata)

# Extract nodes and relationships
survey_nodes = graph_data['survey_nodes']
dataset_nodes = graph_data['dataset_nodes']
variable_nodes = graph_data['variable_nodes']
survey_to_dataset_relationships = graph_data['relationships']['survey_to_dataset']
dataset_to_variable_relationships = graph_data['relationships']['dataset_to_variable']

# Save to CSV (optional)
survey_nodes.to_csv('./data/survey_nodes_debug.csv', index=False)
dataset_nodes.to_csv('./data/dataset_nodes_debug.csv', index=False)
variable_nodes.to_csv('./data/variable_nodes_debug.csv', index=False)
survey_to_dataset_relationships.to_csv('./data/survey_to_dataset_relationships_debug.csv', index=False)
dataset_to_variable_relationships.to_csv('./data/dataset_to_variable_relationships_debug.csv', index=False)

print("Debug files saved:")
print("- survey_nodes_debug.csv")
print("- dataset_nodes_debug.csv")
print("- variable_nodes_debug.csv")
print("- survey_to_dataset_relationships_debug.csv")
print("- dataset_to_variable_relationships_debug.csv")


Reloading datasets_metadata and variables_metadata...


/var/folders/s1/yqb3d76d7053f7dyy9fwtkzw0000gp/T/ipykernel_97248/3101690510.py:49: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  variables_metadata = pd.read_csv('./data/combined_variables_metadata.csv')


Metadata loaded.
Survey Nodes: 298
Dataset Nodes: 3954
Variable Nodes: 2693973
Survey-to-Dataset Relationships: 3954
Dataset-to-Variable Relationships: 2693973
Debug files saved:
- survey_nodes_debug.csv
- dataset_nodes_debug.csv
- variable_nodes_debug.csv
- survey_to_dataset_relationships_debug.csv
- dataset_to_variable_relationships_debug.csv


In [80]:
# Unique top-level survey names
unique_surveys = datasets_metadata['dataset_name'].str.split('/').str[0].unique()
print("Unique Surveys:")
for survey in unique_surveys:
    print(survey)

# Count occurrences of each survey
survey_counts = datasets_metadata['dataset_name'].str.split('/').str[0].value_counts()
print("\nSurvey Counts:")
print(survey_counts)

Unique Surveys:
cps
basic
jun
cbp
zbp
mar
apr
pep
int_charagegroups
aug
int_natcivpop
int_natresafo
dec
int_natrespop
may
ewks
jan
jul
feb
nov
oct
sep
nonemp
int_charage
int_housingunits
int_natmonthly
int_population
surname
acs
acs1
cprofile
pums
acs5
plnat
profile
subject
flows
cd113
cd115
sf1
sf2
ecnbridge2
ecnadmben
ecnbranddeal
ecnbridge1
ecncashadv
ecnbrordeal
ecnccard
ecninvval
ecnclcust
ecnipa
ecncomm
ecnkob
ecncomp
ecnlabor
ecnfran
ecnconact
ecnconcess
ecnlifomfg
ecncrfin
ecnlifomine
ecngrant
ecndissmed
ecnempfunc
ecnlifoval
ecnentsup
ecnlines
ecnguest
ecneoyinv
ecneoyinvwh
ecnloan
ecnequip
ecnlocmfg
ecnguestsize
ecnexpnrg
ecnexpsvc
ecnlocmine
ecnflspace
ecnmargin
ecnfoodsvc
ecnhosp
ecnmatfuel
ecnmealcost
ecnmenutype
ecnpatient
ecnpetrfac
ecnpetrprod
ecnpetrrec
ecnpetrstat
ecnprofit
ecnpurelec
ecnpurmode
ecnrdacq
ecnrdofc
ecnseat
ecnsize
ecnsocial
ecntype
ecntypop
ecnvalcon
sbo
cscbo
popproj
births
deaths
nim
pop
pubschlfin
cre
cscb
language
cochar5
cochar6
cty
housing
monthly

In [81]:
# Define surveys to remove
surveys_to_remove = [
    'basic', 'jan', 'feb', 'mar', 'apr', 'may', 
    'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'
]

# Filter out the redundant entries
datasets_metadata = datasets_metadata[
    ~datasets_metadata['dataset_name'].str.split('/').str[0].isin(surveys_to_remove)
]

# Recheck the unique surveys after removal
remaining_surveys = datasets_metadata['dataset_name'].str.split('/').str[0].unique()
print("\nRemaining Surveys After Filtering:")
print(remaining_surveys)

# Save the cleaned dataset for further inspection
datasets_metadata.to_csv('./data/cleaned_datasets_metadata.csv', index=False)
print("\nFiltered datasets saved to './data/cleaned_datasets_metadata.csv'")



Remaining Surveys After Filtering:
['cps' 'cbp' 'zbp' 'pep' 'int_charagegroups' 'int_natcivpop'
 'int_natresafo' 'int_natrespop' 'ewks' 'nonemp' 'int_charage'
 'int_housingunits' 'int_natmonthly' 'int_population' 'surname' 'acs'
 'acs1' 'cprofile' 'pums' 'acs5' 'plnat' 'profile' 'subject' 'flows'
 'cd113' 'cd115' 'sf1' 'sf2' 'ecnbridge2' 'ecnadmben' 'ecnbranddeal'
 'ecnbridge1' 'ecncashadv' 'ecnbrordeal' 'ecnccard' 'ecninvval'
 'ecnclcust' 'ecnipa' 'ecncomm' 'ecnkob' 'ecncomp' 'ecnlabor' 'ecnfran'
 'ecnconact' 'ecnconcess' 'ecnlifomfg' 'ecncrfin' 'ecnlifomine' 'ecngrant'
 'ecndissmed' 'ecnempfunc' 'ecnlifoval' 'ecnentsup' 'ecnlines' 'ecnguest'
 'ecneoyinv' 'ecneoyinvwh' 'ecnloan' 'ecnequip' 'ecnlocmfg' 'ecnguestsize'
 'ecnexpnrg' 'ecnexpsvc' 'ecnlocmine' 'ecnflspace' 'ecnmargin'
 'ecnfoodsvc' 'ecnhosp' 'ecnmatfuel' 'ecnmealcost' 'ecnmenutype'
 'ecnpatient' 'ecnpetrfac' 'ecnpetrprod' 'ecnpetrrec' 'ecnpetrstat'
 'ecnprofit' 'ecnpurelec' 'ecnpurmode' 'ecnrdacq' 'ecnrdofc' 'ecnseat'
 'ecn

In [82]:
# Ensure `parent_survey` column exists in the datasets_metadata
datasets_metadata['parent_survey'] = datasets_metadata['dataset_name'].str.split('/').str[0]

# Filter rows where parent_survey equals dataset_name
datasets_metadata = datasets_metadata[datasets_metadata['parent_survey'] == datasets_metadata['dataset_name']]

# Check the resulting DataFrame
print("\nFiltered Datasets Metadata:")
print(datasets_metadata.head())

# Save the filtered dataset to a new file for further inspection
datasets_metadata.to_csv('./data/cleaned_datasets_metadata.csv', index=False)
print("\nFiltered datasets saved to './data/cleaned_datasets_metadata.csv'")



Filtered Datasets Metadata:
  dataset_name  year                                              title  \
0          cps  1994  Jun 1994 Current Population Survey: Basic Monthly   
3          cbp  1986   1986 County Business Patterns: Business Patterns   
4          zbp  1994  1994 County Business Patterns - Zip Code Busin...   
5          cbp  1987   1987 County Business Patterns: Business Patterns   
6          cbp  1995   1995 County Business Patterns: Business Patterns   

                                         description  \
0  To provide estimates of employment, unemployme...   
3  County Business Patterns (CBP) is an annual se...   
4  ZIP Code Business Patterns (ZBP) is an annual ...   
5  County Business Patterns (CBP) is an annual se...   
6  County Business Patterns (CBP) is an annual se...   

                                      identifier    contact access_level  \
0  https://api.census.gov/data/id/CPSBASIC199406  CPS Staff       public   
3          http://api.census.go

# 4. Connect to Neo4j
- This cell sets up the Neo4j connection.

In [48]:
from neo4j import GraphDatabase

# Initialize Neo4j connection (replace with your credentials)
neo4j_uri = "bolt://localhost:7687"  # Update with your Neo4j URI
neo4j_user = "neo4j"
neo4j_password = "password"  # Update with your password

try:
    # Create a driver instance
    driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
    
    # Test connection by opening a session and executing a simple query
    with driver.session() as session:
        session.run("RETURN 1")  # Simple query to check connection
    print("Connection successful.")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful.


# 5. Ingest Data into Neo4j
- This cell contains the ingestion logic.

## Clear the Neo4J dB if necessary
To delete all the data in your Neo4j database, you can use the following Cypher query, which will remove all nodes and relationships: \
> MATCH (n) \
> DETACH DELETE n

### Explanation:
- MATCH (n): This matches all nodes in the graph.
- DETACH DELETE n: This deletes the nodes and any relationships attached to them.

### How to Run:
1. Open your Neo4j browser or a Neo4j client.
1. Paste the query and execute it.

This will completely clear your Neo4j database of all nodes and relationships, giving you a fresh starting point for your new data ingestion.



In [68]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm

# Initialize Neo4j connection
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "password"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Load data from CSV files
print("Creating DataFrames...")
survey_nodes = pd.read_csv('./data/survey_nodes.csv')
dataset_nodes = pd.read_csv('./data/dataset_nodes.csv')
variable_nodes = pd.read_csv('./data/variable_nodes.csv')
survey_to_dataset_relationships = pd.read_csv('./data/survey_to_dataset_relationships.csv')
dataset_to_variable_relationships = pd.read_csv('./data/dataset_to_variable_relationships.csv')
print("DataFrames created. Starting Neo4j loading process...")

# Function to load nodes in batches with progress bar
def load_nodes_in_batches(driver, df, query, desc, batch_size=1000):
    total_batches = (len(df) + batch_size - 1) // batch_size
    with tqdm(total=len(df), desc=desc) as pbar:
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i + batch_size]
            with driver.session() as session:
                for _, row in batch.iterrows():
                    session.run(query, **row.to_dict())
            pbar.update(len(batch))

# Function to load relationships in batches with progress bar
def load_relationships_in_batches(driver, df, query, desc, batch_size=1000):
    total_batches = (len(df) + batch_size - 1) // batch_size
    with tqdm(total=len(df), desc=desc) as pbar:
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i + batch_size]
            with driver.session() as session:
                for _, row in batch.iterrows():
                    session.run(query, **row.to_dict())
            pbar.update(len(batch))

# Clear existing data
with driver.session() as session:
    print("Clearing existing data...")
    session.run("MATCH (n) DETACH DELETE n")
print("Existing data cleared.")

# Load Survey nodes
load_nodes_in_batches(
    driver,
    survey_nodes,
    """
    MERGE (s:Survey {survey: $survey})
    """,
    "Loading Survey Nodes"
)

# Load Dataset nodes
load_nodes_in_batches(
    driver,
    dataset_nodes,
    """
    MERGE (d:Dataset {dataset_id: $dataset_id})
    SET d.year = $year, d.month = $month, d.title = $title, d.description = $description
    """,
    "Loading Dataset Nodes"
)

# Load Variable nodes
load_nodes_in_batches(
    driver,
    variable_nodes,
    """
    MERGE (v:Variable {variable_name: $variable_name})
    SET v.label = $label, v.concept = $concept
    """,
    "Loading Variable Nodes"
)

# Create Survey -> Dataset relationships
load_relationships_in_batches(
    driver,
    survey_to_dataset_relationships,
    """
    MATCH (s:Survey {survey: $parent_survey})
    MATCH (d:Dataset {dataset_id: $dataset_id})
    MERGE (s)-[:HAS_DATASET]->(d)
    """,
    "Creating Survey -> Dataset Relationships"
)

# Create Dataset -> Variable relationships
load_relationships_in_batches(
    driver,
    dataset_to_variable_relationships,
    """
    MATCH (d:Dataset {dataset_id: $dataset_name})
    MATCH (v:Variable {variable_name: $variable_name})
    MERGE (d)-[:HAS_VARIABLE]->(v)
    """,
    "Creating Dataset -> Variable Relationships"
)

print("All data loaded into Neo4j successfully!")

Creating DataFrames...
DataFrames created. Starting Neo4j loading process...
Clearing existing data...
Existing data cleared.


Loading Variable Nodes:   0%|         | 7000/2693973 [01:03<6:44:00, 110.85it/s]


KeyboardInterrupt: 

# Step 6: Securely Loading OpenAI API Key and Using LLM for Concept Extraction

The goal of this step is to securely load the OpenAI API key from a `.env` file and utilize the LLM (Large Language Model) to enhance the knowledge graph. By extracting key concepts and terms from variable descriptions, we enrich the graph with semantic information.

**Key Steps:**
1. **Loading the API Key**: 
    - We load the OpenAI API key securely from a `.env` file using the `python-dotenv` package. This avoids hardcoding sensitive credentials in the source code, ensuring better security and flexibility.
   
2. **Using LLM for Concept Extraction**:
    - Once the API key is loaded, we use OpenAI’s GPT-based model to process the variable descriptions. The model extracts key concepts, terms, and entities from the descriptions (e.g., "income," "education level"), which we can then use to create new concept nodes in the knowledge graph.
   
3. **Enhancing the Knowledge Graph**:
    - After extracting the concepts, we create **concept nodes** and link them to the relevant **variables** using `:MEASURES` relationships. We can also link surveys that cover similar concepts, allowing us to analyze relationships between datasets based on the concepts they measure.

This approach leverages the power of GPT to enhance the graph beyond simple variable names, creating a richer, more semantically aware dataset that will be useful for querying, analysis, and discovering relationships that were not obvious at first glance.

## 6.1 Loading the API Key


In [ ]:
import openai
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")

## 6.2 Testing functionality/connectivity of the GPT

### Extracting Key Concepts with ChatGPT
This is for testing, to be sure things are working. In this step, we'll define a function that uses OpenAI's GPT model to process variable descriptions and extract important concepts, keywords, or themes. The extracted concepts will help us create concept nodes in the knowledge graph.

### Here’s the process:
1. Input: We'll pass the variable descriptions to the GPT model via the OpenAI API.
1. Output: The model will return key concepts or keywords that are semantically related to the descriptions.
1. Linking: These concepts will be used to enrich the knowledge graph, creating concept nodes and linking them to the variables.

In [ ]:
import openai

def extract_concepts_from_description(description):
    """
    Extract key concepts from a description using OpenAI's API
    
    Args:
        description (str): The text description to analyze
    
    Returns:
        str: Extracted concepts and terms
    """
    try:
        client = openai.OpenAI()  # Uses OPENAI_API_KEY from environment
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Can upgrade to gpt-4 if available
            messages=[
                {"role": "system", "content": "You are a helpful assistant specializing in concept extraction."},
                {"role": "user", "content": f"Extract key concepts and terms from this variable description: {description}"}
            ],
            max_tokens=100,
            temperature=0.5
        )
        
        # Extract concepts from the response
        concepts = response.choices[0].message.content.strip()
        
        return concepts
    
    except Exception as e:
        print(f"Error in concept extraction: {e}")
        return None

# Example usage
if __name__ == "__main__":
    sample_description = "Flag for Production workers average for year"
    concepts = extract_concepts_from_description(sample_description)
    print("Extracted Concepts:", concepts)

  

## Step 6.3: Link Variables to Concepts

The goal here is to take the concepts extracted from the variable descriptions and link them to their corresponding variables in the Neo4j knowledge graph.


In [ ]:
import openai
import pandas as pd
import logging
import os

# Configure logging
log_file_path = 'concept_extraction.log'
# Ensure the log file is cleared each time
if os.path.exists(log_file_path):
    os.remove(log_file_path)

logging.basicConfig(
    filename=log_file_path,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s'
)

# Function to extract concepts from the variable description using OpenAI
def extract_concepts_from_description(description):
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo",  # You can upgrade to gpt-4 if desired
            prompt=f"Extract key concepts and terms from this variable description: {description}",
            max_tokens=100,  # Limit tokens to get a concise response
            n=1,
            stop=None,
            temperature=0.3
        )
        concepts = response.choices[0].text.strip()
        return concepts
    except Exception as e:
        logging.error(f"Error in concept extraction for description '{description}': {e}")
        return None

# Main extraction process
def extract_missing_concepts(child_to_variable):
    logging.info("Starting concept extraction for missing entries")
    
    for index, row in child_to_variable.iterrows():
        if pd.isna(row['concept']) or row['concept'] == '':  # If concept is missing
            logging.info(f"Attempting to extract concept for variable {row['variable_name']}")
            
            # Attempt to extract the concept using the variable's label or description
            description = row['label']  # Or use another column if description is separate
            extracted_concepts = extract_concepts_from_description(description)
            
            if extracted_concepts:
                logging.info(f"Successfully linked Variable: {row['variable_name']} to Concept: {extracted_concepts}")
                child_to_variable.at[index, 'concept'] = extracted_concepts  # Assign the extracted concept
            else:
                logging.warning(f"Failed to extract concept for {row['variable_name']}")
    
    logging.info("Concept extraction process completed")
    return child_to_variable

# Actual execution
if __name__ == "__main__":
    # Ensure OpenAI API key is set
    if not os.getenv('OPENAI_API_KEY'):
        logging.error("OpenAI API key not set. Please set the OPENAI_API_KEY environment variable.")
        raise ValueError("OpenAI API key is required")

    # Process all data
    updated_child_to_variable = extract_missing_concepts(child_to_variable)
    
    # Save the updated DataFrame
    updated_child_to_variable.to_csv('updated_child_to_variable.csv', index=False)
    
    # Display updated entries
    print("Concepts successfully populated for missing entries:")
    print(updated_child_to_variable.head())

In [ ]:
print(child_to_variable[child_to_variable['variable_name'] == 'EMPAVPW_F'])
